In [ ]:
import ee
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Inisialisasi Earth Engine
ee.Initialize()

# Lokasi titik
coordinates = np.array([116.2653, -1.8789])
x = float(coordinates[0])
y = float(coordinates[1])
location_point = ee.Geometry.Point(x, y)

# Skala dan tanggal
scale = 1000
date_start = '2022-01-01'
date_end = '2025-04-30'

# Fungsi bantu untuk membuat daftar tanggal harian
def date_range(start, end):
    start_dt = datetime.strptime(start, "%Y-%m-%d")
    end_dt = datetime.strptime(end, "%Y-%m-%d")
    while start_dt <= end_dt:
        yield start_dt
        start_dt += timedelta(days=1)

# List untuk menyimpan hasil
results = []

# Loop harian
for day in date_range(date_start, date_end):
    start = day.strftime('%Y-%m-%d')
    end = (day + timedelta(days=1)).strftime('%Y-%m-%d')

    # Filter dan agregasi per hari
    daily_sum = ee.ImageCollection("ECMWF/ERA5/HOURLY") \
        .filterDate(start, end) \
        .filterBounds(location_point) \
        .select("total_precipitation") \
        .sum() \
        .multiply(1000)  # m ke mm

    # Ambil nilai dari titik
    sample = daily_sum.sample(
        region=location_point,
        scale=scale,
        numPixels=1,
        geometries=True
    ).getInfo()

    # Ambil hasil jika tersedia
    if sample['features']:
        val = sample['features'][0]['properties']['total_precipitation']
        results.append({
            'date': start,
            'total_precipitation_mm': max(val, 0)  # pastikan tidak negatif
        })

# Buat DataFrame
df = pd.DataFrame(results)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

print(df)


In [ ]:
df.to_csv('dataset_total_prec_per_dayyy.csv')